## Previsão de cotação da empresa WEG SA utilizando redes neurais recorentes RNNs


WEG S.A é uma empresa multinacional brasileira com sede na cidade de Jaraguá do Sul, no estado de Santa Catarina.

<br><img src="WEG.png"><br>

A empresa é uma das maiores fabricantes de equipamentos elétricos do mundo, atuando nas áreas de comando e proteção, variação de velocidade, automação de processos industriais, geração e distribuição de energia e tintas e vernizes industriais, entre outros produtos.

Neste trabalho iremos implementar uma RNNs para realizar a predição diária das cotações da empresa.

O dataset **"WEGE3.SA_5_latest_years.csv"** possui informações dispostas em colunas :

* **Date**:  Data das operações na bolsa (diária)
* **Close**:  Valor de Fechamento das ações (no dia)
* **Open**:  Valor da Abertura das ações (no dia)
* **High**:  Valor máximo das ações (no dia)
* **Low**:  Valor mínimo das ações (no dia)
* **Vol**:  Volume de contratos negociados (no dia)




### Bibliotecas

In [ ]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
import plotly.graph_objects as go 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Carregando os dados

Vamos começar lendo o arquivo WEGE3.SA_5_latest_years.csv em um dataframe do pandas, mas antes vamos dar uma olhadinha no gráfico de variação do último mês da ação WEGE3.

In [ ]:
DataSet=pd.read_csv('WEGE3.SA_latest_30_days.csv')

In [ ]:
fig = go.Figure(data=[go.Candlestick(x=DataSet['Date'],
                open=DataSet['Open'], high=DataSet['High'],
                low=DataSet['Low'], close=DataSet['Close'])
                     ])

fig.update_layout(xaxis_rangeslider_visible=False)  
fig.show()



### Agora vamos ler o arquivo do período desejável 

In [ ]:
DataSet=pd.read_csv('WEGE3.SA_5_latest_years.csv')
DataSet=DataSet.dropna()
DataSet.head()

In [ ]:
DataSet.describe()

### Inicialmente iremos criar uma RNN baseada apenas no Valor de Abertura

In [ ]:
plt.scatter(DataSet['Date'],DataSet['Open'],)
plt.show()

base_treinamento = DataSet.iloc[:, 1:2].values

#DataSet.drop(['Date','Close','High','Low', 'Volume'],axis=1,inplace=True)

In [ ]:
base_treinamento

### Normalizar os dados da ação

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
DataScaled=scaler.fit_transform(base_treinamento)

In [ ]:
plt.scatter(DataSet['Date'],DataScaled,)
plt.show()
print(DataScaled)

### Definição dos previsores

In [ ]:
previsores = []
preco_real = []
NRecursao = 90
DataSetLen = len(DataScaled) #quantidade de dados no dataset
print(DataSetLen)

In [ ]:

for i in range(NRecursao, DataSetLen):
    previsores.append(DataScaled[i-NRecursao:i,0])
    preco_real.append(DataScaled[i,0])

previsores, preco_real = np.array(previsores), np.array(preco_real)

In [ ]:
previsores.shape

### Tranformar para o formato do Tensor do Keras

<br><img src="TensorKeras.png"><br>

In [ ]:
previsores = np.reshape(previsores, (previsores.shape[0], previsores.shape[1], 1))

In [ ]:
previsores.shape

### Estrutura da Rede Neural

In [ ]:
# Camada de entrada
regressor = Sequential()
regressor.add(LSTM(units = 100, return_sequences = True, input_shape = (previsores.shape[1], 1)))
regressor.add(Dropout(0.3))

# Cada Oculta 1
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.3))

# Cada Oculta 2
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.3))

# Cada Oculta 3
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.3))

# Camada de Saída
regressor.add(Dense(units = 1, activation = 'linear'))


### Construindo a Rede

In [ ]:
regressor.compile(optimizer = 'rmsprop', loss = 'mean_squared_error',
                  metrics = ['mean_absolute_error'])
regressor.fit(previsores, preco_real, epochs = 2000, batch_size = 32)

### Conjunto de dados para o Teste

In [ ]:
DataSet_teste=pd.read_csv('WEGE3.SA_latest_30_days.csv')

preco_real_teste = DataSet_teste.iloc[:, 1:2].values

base_completa = pd.concat((DataSet['Open'], DataSet_teste['Open']), axis = 0)
entradas = base_completa[len(base_completa) - len(DataSet_teste) - NRecursao:].values

entradas = entradas.reshape(-1, 1)
entradas = scaler.transform(entradas)

In [ ]:
DataSetTestLen = len(DataSet_teste)
NPredictions = 90

X_teste = []
for i in range(NRecursao, DataSetTestLen+NRecursao):
    X_teste.append(entradas[i-NRecursao:i, 0])
    
X_teste = np.array(X_teste)
X_teste = np.reshape(X_teste, (X_teste.shape[0], X_teste.shape[1], 1))

previsoes = regressor.predict(X_teste)
previsoes = scaler.inverse_transform(previsoes)


In [ ]:
RNN=[]
predictions_teste=X_teste[0].T
predictions_teste=np.reshape(predictions_teste, (predictions_teste.shape[0], predictions_teste.shape[1], 1))

predictions_teste[0][NRecursao-1][0]=regressor.predict(predictions_teste)[0][0]
RNN.append(regressor.predict(predictions_teste)[0])

for i in range(NPredictions-1):
    predictions_teste=np.roll(predictions_teste,-1)
    predictions_teste[0][NRecursao-1][0]=regressor.predict(predictions_teste)[0][0]
    RNN.append(regressor.predict(predictions_teste)[0])
RNN = scaler.inverse_transform(RNN)

print(RNN.mean())
print(previsoes.mean())
print(preco_real_teste.mean())

In [ ]:
plt.plot(preco_real_teste, color = 'red', label = 'Preço real')
plt.plot(previsoes, color = 'blue', label = 'Previsões')
#plt.plot(RNN, color = 'green', label = 'RNN')

plt.title('WEGE3')
plt.xlabel('Tempo')
plt.ylabel('Valor')
plt.legend()
plt.show()